In [9]:
import albumentations as A
import matplotlib.pyplot as plt
import torch
import cv2
from matplotlib.patches import Rectangle
import os
import PIL 
import numpy as np
# rm -rf `find -type d -name .ipynb_checkpoints`

In [10]:
def generate_transform(width, height): 
    h = int(height * np.random.uniform(0.5, 1))
    w = int(width * np.random.uniform(0.5, 1))

    return A.Compose([
        A.RandomCrop(width=w, height=h),
        A.RandomBrightnessContrast(p=0.2),
        A.OneOf([
                              A.HorizontalFlip(p=0.5),
                              A.MotionBlur(p=0.5),            
        ], p=1),
        A.OneOf([
                              A.OpticalDistortion(p=0.5),
                              A.GaussNoise(p=0.5)                 
        ], p=1),

    ],  bbox_params=A.BboxParams(format='yolo', min_area=1024, min_visibility=0.1, label_fields=['class_labels']))

In [11]:
path = 'African_Wildlife_augmented/'
fnames = os.listdir(f'{path}/train')
fnum = set(f.split('.')[0] for f in fnames) # number of image
num_img = len(os.listdir(f'{path}/annotations')) 
for i, f in enumerate(fnum): 
    image_path = f"{path}/train/{f}.jpg"
    label_path = f"{path}/annotations/{f}.txt"
    
    # prepare images for transform
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:-1]

    # prepare bboxes for transform
    bounding_boxes = []
    with open(label_path) as f: # open the image 
        for label in f.readlines():
            class_label, x, y, width, height = [
                float(x) if float(x) != int(float(x)) else int(x)
                for x in label.replace("\n", "").split()
            ]

            bounding_boxes.append([class_label, x, y, width, height])

    # separate bboxes from class labels for transform
    bboxes = []
    class_labels = []
    for idx in range(len(bounding_boxes)): 
        bboxes.append(bounding_boxes[idx][1:])
        class_labels.append(bounding_boxes[idx][:1])

    # apply transforms
    transform = generate_transform(w, h)
    transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    transformed_class_labels = transformed['class_labels']

    # join class and bboxes again 
    transformed_labels = []
    for idx, b in enumerate(transformed_bboxes): 
        box = list(b)
        box.insert(0,transformed_class_labels[idx][0])
        box = ''.join([str(x) + ' ' for x in box])
        transformed_labels.append(box)
        transformed_labels.append('\n')
    transformed_labels = ''.join(transformed_labels)

    name = str(int(i)+num_img)
    new_image = PIL.Image.fromarray(transformed_image)
    new_image.save(f'{path}/train/{name}.jpg')

    with open(f'{path}/annotations/{name}.txt', "w") as output:
        output.write(str(transformed_labels))




ValueError: Expected x_min for bbox (-5.000000000143778e-07, 0.2229725, 0.2867185, 0.5418915, [3]) to be in the range [0.0, 1.0], got -5.000000000143778e-07.

In [21]:
path = 'African_Wildlife_augmented/'
fnames = os.listdir(f'{path}/train')
tr = set(f.split('.')[0] for f in fnames) # number of image

names = os.listdir(f'{path}/annotations')
ann = set(f.split('.')[0] for f in names) # number of image
a = [x for x in ann if x not in tr]

len(fnames), len(names), len(names) - len(fnames)

(1543, 2144, 601)